In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, CuDNNLSTM, Flatten
import math
from sklearn.metrics import mean_squared_error

#Compile with an inital model with 2 LSTMs of 64 units:
model = Sequential()
model.add(CuDNNLSTM(units=64,return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(CuDNNLSTM(units=64))
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer='rmsprop',loss='mean_squared_error')
model.fit(X_train,y_train,epochs=20,batch_size=16)

#Trying 4*3*3=36 different combinations of LSTM cells and dense layers:

dense_layers = [0, 1, 2, 3]
units = [32, 64, 128]
n_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for unit in units:
        for layers in n_layers:
            NAME = "{}-lstm-{}-units-{}-dense-{}".format(layers, unit, dense_layer, int(time.time()))
            print(NAME)
            
            model = Sequential()

            model.add(CuDNNLSTM(units=unit, return_sequences= layers-1>0, input_shape=(X_train.shape[1],1)))

            for l in range(layers-1):
                model.add(CuDNNLSTM(units=unit , return_sequences= l < layers-2 ))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(unit))

            model.add(Dense(1))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
            
            
            # Compiling the RNN
            model.compile(optimizer='rmsprop',loss='mean_squared_error', metrics=['MeanAbsoluteError'])
            # Fitting to the training set
            model.fit(X_train,y_train,epochs=10,batch_size=32, validation_split=0.3,callbacks=[tensorboard])

#Compile the optimzed model based on the results:
model.compile(optimizer='rmsprop',loss='mean_squared_error')
model.fit(X_train,y_train,epochs=50,batch_size=32)